### 2．学習データの準備

In [ ]:
# PyTorchライブラリの読み込み
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# gzipライブラリの読み込み
import gzip
# Numpyライブラリの読み込み
import numpy as np
# Pandasライブラリの読み込み
import pandas as pd
# Scikit-learnライブラリの読み込み
from sklearn import datasets, model_selection

# Matplotlibライブラリの読み込み
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
# FASHION-MNISTデータセットの読込
with gzip.open('./data/fashion-mnist/train-images-idx3-ubyte.gz', 'rb') as f:
    mnist_data = np.frombuffer(f.read(), np.uint8, offset=16)
    mnist_data = mnist_data.reshape(-1, 784)

In [ ]:
# 説明変数を正規化して格納
mnist_data = mnist_data / 255

# データフレーム形式で説明変数を表示
pd.DataFrame(mnist_data)

In [ ]:
# 1枚目の画像を表示
plt.imshow(mnist_data[0].reshape(28, 28), cmap=cm.gray_r)
plt.show()

In [ ]:
# 目的変数の格納と表示
with gzip.open('./data/fashion-mnist/train-labels-idx1-ubyte.gz', 'rb') as f:
    mnist_label = np.frombuffer(f.read(), np.uint8, offset=8)

mnist_label

In [ ]:
# 訓練データのサイズ
train_size = 5000
# テストデータのサイズ
test_size = 500

# データセットを訓練用とテスト用に分割
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(
    mnist_data, mnist_label, train_size=train_size,test_size=test_size)

In [ ]:
# 784の1次元配列を28×28の2次元配列へリサイズ
train_X = train_X.reshape((len(train_X), 1, 28, 28))
test_X = test_X.reshape((len(test_X), 1, 28, 28))

### 3．テンソルの作成

In [ ]:
# 訓練用のテンソル作成
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

# テスト用のテンソル作成
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

# テンソルのサイズを表示
print(train_X.shape)
print(train_Y.shape)

In [ ]:
# 説明変数と目的変数のテンソルをまとめる
train = TensorDataset(train_X, train_Y)

# 1つ目のテンソルを確認
print(train[0])

# ミニバッチに分ける
train_loader = DataLoader(train, batch_size=100, shuffle=True)

### 4．ネットワークの作成

In [ ]:
# ネットワークの作成
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 畳み込み層
        self.conv1 = nn.Conv2d(1, 6, 5) # 入力チャンネル数、出力チャンネル数、フィルタサイズ
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 全結合層
        self.fc1 = nn.Linear(256, 64) # 256 = (((28-5+1)/2 )-5+1)/2 * (((28-5+1)/2 )-5+1)/2 * 16
        self.fc2 = nn.Linear(64, 10)
    
    def forward(self, x):
        # プーリング層
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) # 領域サイズ
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 256)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)
    
# インスタンスの生成
model = Net()

### 5．モデルの学習

In [ ]:
# 誤差関数のセット
criterion = nn.CrossEntropyLoss()

# 最適化関数のセット
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 学習開始
for epoch in range(1000):
    total_loss = 0
    # 分割したデータの取り出し
    for train_x, train_y in train_loader:
        # 計算グラフの構築
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 勾配をリセットする
        optimizer.zero_grad()
        # 順伝播の計算
        output = model(train_x)
        # 誤差の計算
        loss = criterion(output, train_y)
        # 逆伝播の計算
        loss.backward()
        # 重みの更新
        optimizer.step()
        # 誤差の累積
        total_loss += loss.data[0]
    # 累積誤差を100回ごとに表示    
    if (epoch+1) % 100 == 0:
        print(epoch+1, total_loss)

In [ ]:
# 計算グラフの構築
test_x, test_y = Variable(test_X), Variable(test_Y)
# 出力が0と1のどちらか
result = torch.max(model(test_x).data, 1)[1]
# モデルの精度を計算
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# モデルの精度を表示
accuracy